In [ ]:
# notebooks/audit_log_exploracao.ipynb

import pandas as pd
import plotly.express as px
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Inicia uma sessão Spark para LEITURA
builder = SparkSession.builder.appName("AnaliseGold") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = builder.getOrCreate()

# Carrega os dados diretamente da camada Gold
df_apps = spark.read.format("delta").load("../data/gold/apps_prioritarios")
df_funil = spark.read.format("delta").load("../data/gold/funil_governanca")

print("Tabela de Funil de Governança")
df_funil.show()

print("\nAmostra dos Aplicativos Prioritários")
df_apps.select("admin_displayname", "usuarios_unicos", "total_sessoes").orderBy(col("usuarios_unicos").desc()).show()

# Exemplo de visualização com Plotly
df_funil_pd = df_funil.toPandas()
fig = px.funnel(df_funil_pd, x='Quantidade', y='Etapa do Funil', title='Funil de Governança de Aplicativos')
fig.show()

spark.stop()